In [117]:
import pandas as pd
import rapidfuzz
from joblib import Parallel, delayed
from tqdm import tqdm
import re
import awswrangler as wr
import boto3

In [118]:
def pre_match_to_similar_text(text,
                              list_of_target_text,
                              similarity_threshold) -> str:
    
    result = rapidfuzz.process.extract(text,
                                       list_of_target_text,
                                       scorer=rapidfuzz.fuzz.token_set_ratio,
                                       limit=1,
                                       score_cutoff=similarity_threshold)

    return result[0][0] if (len(result) > 0) else 'none'


def pre_grouping_text(df,
                      column_string,
                      percent_threshold,
                      similarity_threshold,
                      new_column_name):
 

    if new_column_name is None:
        new_column_name = column_string

    threshold_to_be_major = percent_threshold / 100 * len(df[column_string].unique())


    t = df[column_string].value_counts().to_frame()
    major_group_list = df[df[column_string].isin(t[t[column_string] > threshold_to_be_major].index)][column_string].unique()
    minor_group_list = df[~df[column_string].isin(major_group_list)][column_string].tolist()

    grouping_dictionary = {major_group_list[i]: major_group_list[i] for i in range(len(major_group_list))}

    result = Parallel(n_jobs=10)(delayed(pre_match_to_similar_text)(
        minor_group, major_group_list, similarity_threshold) for minor_group in tqdm(minor_group_list))
    temp = dict(zip(minor_group_list, result))

    grouping_dictionary.update(temp)
    df[new_column_name] = df[column_string].map(grouping_dictionary)

    return df


def pre_remove_position(text: str) -> str:
  
    for direction in ['lh', 'rh', 'right', 'left', 'upper']:
        text = text.lower()
        text = re.sub(direction, '', text)

    return text.strip()

In [119]:
wr.config.s3_endpoint_url = "http://192.168.1.4:8333"
bucket = "scope_part"

partDf = wr.s3.read_parquet(f"s3://{bucket}/")
partDf

,CaseID,Description,fVAL,DamageCond,CoType
0,12657878,bonnet,550.0,bent/dented,I
1,12657878,bumper front cover,350.0,creased/torn,I
2,12657878,bumper front lower grille,110.0,bracket torn,I
3,12657878,bumper reinforcement front,250.0,bent/dented,I
4,12657878,fog lamp front rh,150.0,bracket snapped,I
...,...,...,...,...,...
12453795,12908158,fuse / relay box,300.0,broken,I
12453796,12908158,airbag rotary coupling,500.0,activated,I
12453797,12908162,windscreen glass - front,309.2,,I
12453798,12908163,tint film,280.0,,I


In [120]:
wr.config.s3_endpoint_url = "http://192.168.1.4:8333"
bucket2 = "scope_case"

caseDf = wr.s3.read_parquet(f"s3://{bucket2}/", columns=["CaseID"])

In [121]:
partDf = partDf.merge(caseDf, on="CaseID")

In [122]:
# partDf["Description"].value_counts().head(100).reset_index()

In [123]:
fuzzyMatchDf = pre_grouping_text(partDf,
                         column_string='Description',
                         percent_threshold=3,
                         similarity_threshold=60,
                         new_column_name='FuzzyMatchDesc')

100%|██████████| 2800303/2800303 [01:01<00:00, 45544.65it/s]


In [124]:
fuzzyMatchDf["FuzzyMatchDesc"].value_counts().to_frame()["FuzzyMatchDesc"].sum()

7186271

In [125]:
fuzzyMatchDf["FuzzyMatchDesc"].value_counts().to_frame().head(50)["FuzzyMatchDesc"].sum()

5563013

In [126]:
mappingDf = fuzzyMatchDf["FuzzyMatchDesc"].value_counts().reset_index().rename(columns={"index" : "part", "FuzzyMatchDesc" : "count"})

In [127]:
fuzzyMatchDf[fuzzyMatchDf["FuzzyMatchDesc"] == "none"]["Description"].value_counts().reset_index().head(20)

,index,Description
0,oil cooler gearbox,7576
1,bodyside rh,5909
2,pas oil cooler,5175
3,bodyside lh,5006
4,pas pump,4505
5,horn,4337
6,side step lh,4255
7,fuse box,4183
8,steering tie rod end rh,4141
9,steering tie rod end lh,3867


In [128]:
mappingDf.to_csv("../../data/tmp/mapping.csv")

In [129]:
semanticGroupDf = pd.read_csv("/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/tmp/complete_view_mapping.csv")

In [130]:
semanticGroupDf.drop(columns=["Unnamed: 0.1", "Unnamed: 0"], inplace=True)

In [131]:
semanticGroupDf

,lvl_1_desc,count,lvl_2_desc,lvl_3_desc,front_view,front_view_left,front_view_right,rear_view,rear_view_left,rear_view_right
0,bonnet insulation,17505,bonnet,bonnet,1,1,1,0,0,0
1,bonnet hinge lh,17339,bonnet,bonnet,1,1,1,0,0,0
2,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
3,bonnet hinge rh,18246,bonnet,bonnet,1,1,1,0,0,0
4,bonnet moulding,24143,bonnet,bonnet,1,1,1,0,0,0
5,bonnet lock,14106,bonnet,bonnet,1,1,1,0,0,0
6,bumper front impact pad,12837,bumper front,bumper front,1,1,1,0,0,0
7,"face, fr. bumper",62882,bumper front,bumper front,1,1,1,0,0,0
8,bumper spoiler front,64004,bumper front,bumper front,1,1,1,0,0,0
9,front bumper,149934,bumper front,bumper front,1,1,1,0,0,0


In [132]:
semanticGroupDf["lvl_2_desc"] = semanticGroupDf["lvl_2_desc"].str.strip()
semanticGroupDf["lvl_3_desc"] = semanticGroupDf["lvl_3_desc"].str.strip()

lvl2LabelDistribDf = semanticGroupDf.groupby('lvl_2_desc')["count"].sum().reset_index()
lvl3LabelDistribDf = semanticGroupDf.groupby('lvl_3_desc')["count"].sum().reset_index()


In [133]:
len(lvl2LabelDistribDf)

34

In [134]:
len(lvl3LabelDistribDf)

25

In [135]:
import plotly.express as px
px.bar(lvl2LabelDistribDf, x="lvl_2_desc", y="count")

In [136]:
px.bar(lvl3LabelDistribDf, x="lvl_3_desc", y="count")

In [137]:
partlistLabelDf = pd.merge(left=fuzzyMatchDf,  left_on="FuzzyMatchDesc",right=semanticGroupDf,right_on="lvl_1_desc")
partlistLabelDf 

,CaseID,Description,fVAL,DamageCond,CoType,FuzzyMatchDesc,lvl_1_desc,count,lvl_2_desc,lvl_3_desc,front_view,front_view_left,front_view_right,rear_view,rear_view_left,rear_view_right
0,12657878,bonnet,550.00,bent/dented,I,bonnet,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
1,12657888,bonnet,510.44,bent,I,bonnet,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
2,12657890,bonnet,504.94,dented,I,bonnet,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
3,12657900,bonnet,450.00,crumpled,I,bonnet,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
4,12657903,bonnet,280.00,bent,I,bonnet,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7186266,12606480,rear panel inner trim,250.00,broken,I,rear panel inner trim,rear panel inner trim,12963,rear panel,rear panel,0,0,0,1,1,1
7186267,12606779,firewall rear trim,225.00,,I,rear panel inner trim,rear panel inner trim,12963,rear panel,rear panel,0,0,0,1,1,1
7186268,12607452,inner carrier rh,384.00,holder torn,I,rear panel inner trim,rear panel inner trim,12963,rear panel,rear panel,0,0,0,1,1,1
7186269,12607489,taillamp inner lh n rh,691.00,broken,I,rear panel inner trim,rear panel inner trim,12963,rear panel,rear panel,0,0,0,1,1,1


In [138]:
filterDmgCond = ["housing", "holder", "bracket"]


In [139]:
partlistLabelDf = partlistLabelDf[~partlistLabelDf["DamageCond"].str.contains("|".join(filterDmgCond))]
partlistLabelDf

,CaseID,Description,fVAL,DamageCond,CoType,FuzzyMatchDesc,lvl_1_desc,count,lvl_2_desc,lvl_3_desc,front_view,front_view_left,front_view_right,rear_view,rear_view_left,rear_view_right
0,12657878,bonnet,550.00,bent/dented,I,bonnet,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
1,12657888,bonnet,510.44,bent,I,bonnet,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
2,12657890,bonnet,504.94,dented,I,bonnet,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
3,12657900,bonnet,450.00,crumpled,I,bonnet,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
4,12657903,bonnet,280.00,bent,I,bonnet,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7186264,12606104,rear panel inner trim,235.00,broken,I,rear panel inner trim,rear panel inner trim,12963,rear panel,rear panel,0,0,0,1,1,1
7186265,12606233,rear panel inner trim,350.00,dented,I,rear panel inner trim,rear panel inner trim,12963,rear panel,rear panel,0,0,0,1,1,1
7186266,12606480,rear panel inner trim,250.00,broken,I,rear panel inner trim,rear panel inner trim,12963,rear panel,rear panel,0,0,0,1,1,1
7186267,12606779,firewall rear trim,225.00,,I,rear panel inner trim,rear panel inner trim,12963,rear panel,rear panel,0,0,0,1,1,1


In [140]:
checkDescDf = partlistLabelDf.groupby(["lvl_2_desc", "Description"])["DamageCond"].count().reset_index().rename(columns={
    "DamageCond" : "count"
})
checkDescDf.sort_values(by=["count"], inplace=True, ascending=False)


In [141]:
for desc in checkDescDf["lvl_2_desc"].unique().tolist():
    mostCommonDesc = checkDescDf[checkDescDf["lvl_2_desc"] == desc].head(20)
    print(mostCommonDesc)

         lvl_2_desc                       Description   count
15962  bumper front                bumper front cover  311208
20908  bumper front        bumper reinforcement front   91224
21288  bumper front              bumper spoiler front   36856
19440  bumper front       bumper front shock absorber   13501
17412  bumper front          bumper front lower cover   12783
24625  bumper front                      front bumper   12742
18591  bumper front  bumper front reinforcement panel   10669
16996  bumper front           bumper front impact pad   10601
23619  bumper front                  face, fr. bumper    9670
35805  bumper front              stabiliser bar front    7237
23660  bumper front                  face, rr. bumper    6633
15679  bumper front   bumper front bracket support rh    6271
15653  bumper front   bumper front bracket support lh    5827
14917  bumper front           bumper bracket front rh    4947
15248  bumper front                      bumper front    4931
14889  b

In [142]:
remapping = {
    "wheel" : [
        "moulding wheel arch front rh",
        "moulding wheel arch front lh",
        "front wheel arch protector lh",
        "extension wheel arch front rh",
        "extension wheel arch front lh",
        "wheel arch finisher front lh",
        "wheel hub front lh",
        "moulding wheel arch rear rh",
        "extension wheel arch rear rh",
        "wheel house extension rear rh",
        "wheel arch finisher rear rh",
        "wheel hub rear lh",
        "wheel arch finisher rear lh"
        "wheel house extension rear lh"
        "moulding wheel arch rear lh",
        "extension wheel arch rear lh",
        "moulding wheel arch front rh"
    ]
}

In [143]:
for k, v in remapping.items():
    partlistLabelDf.loc[partlistLabelDf["Description"].isin(v) , "lvl_2_desc"] = k
    partlistLabelDf.loc[partlistLabelDf["Description"].isin(v) , "lvl_3_desc"] = k

In [144]:
partlistLabelDf[partlistLabelDf["Description"] == "moulding wheel arch front rh"]

,CaseID,Description,fVAL,DamageCond,CoType,FuzzyMatchDesc,lvl_1_desc,count,lvl_2_desc,lvl_3_desc,front_view,front_view_left,front_view_right,rear_view,rear_view_left,rear_view_right
4819814,12659235,moulding wheel arch front rh,522.80,broken,I,grille front moulding,grille front moulding,102827,wheel,wheel,1,1,1,0,0,0
4819855,12660672,moulding wheel arch front rh,280.00,deep cut,I,grille front moulding,grille front moulding,102827,wheel,wheel,1,1,1,0,0,0
4819863,12660830,moulding wheel arch front rh,297.42,torn,I,grille front moulding,grille front moulding,102827,wheel,wheel,1,1,1,0,0,0
4819911,12663054,moulding wheel arch front rh,250.00,broken,I,grille front moulding,grille front moulding,102827,wheel,wheel,1,1,1,0,0,0
4820070,12667938,moulding wheel arch front rh,522.80,broken,I,grille front moulding,grille front moulding,102827,wheel,wheel,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4918113,12599754,moulding wheel arch front rh,125.30,broken,I,grille front moulding,grille front moulding,102827,wheel,wheel,1,1,1,0,0,0
4918125,12600135,moulding wheel arch front rh,641.00,scratched/cut,I,grille front moulding,grille front moulding,102827,wheel,wheel,1,1,1,0,0,0
4918160,12601485,moulding wheel arch front rh,113.90,stud broken,I,grille front moulding,grille front moulding,102827,wheel,wheel,1,1,1,0,0,0
4918237,12603899,moulding wheel arch front rh,113.90,broken,I,grille front moulding,grille front moulding,102827,wheel,wheel,1,1,1,0,0,0


In [145]:
blacklistComponent = ["weatherstrip", "finisher", "wiring", "hinge", "bulb", "trimboard", "bracket", "sensor", "lock", "sealant", "silencer"]

In [1]:
removePart = [
    "wheel", 
    "door front lh",
    "door front rh",
    "door rear lh",
    "door rear lh",

]

SyntaxError: unterminated string literal (detected at line 1) (2256083413.py, line 1)

In [147]:
partlistLabelDf

,CaseID,Description,fVAL,DamageCond,CoType,FuzzyMatchDesc,lvl_1_desc,count,lvl_2_desc,lvl_3_desc,front_view,front_view_left,front_view_right,rear_view,rear_view_left,rear_view_right
0,12657878,bonnet,550.00,bent/dented,I,bonnet,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
1,12657888,bonnet,510.44,bent,I,bonnet,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
2,12657890,bonnet,504.94,dented,I,bonnet,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
3,12657900,bonnet,450.00,crumpled,I,bonnet,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
4,12657903,bonnet,280.00,bent,I,bonnet,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7186264,12606104,rear panel inner trim,235.00,broken,I,rear panel inner trim,rear panel inner trim,12963,rear panel,rear panel,0,0,0,1,1,1
7186265,12606233,rear panel inner trim,350.00,dented,I,rear panel inner trim,rear panel inner trim,12963,rear panel,rear panel,0,0,0,1,1,1
7186266,12606480,rear panel inner trim,250.00,broken,I,rear panel inner trim,rear panel inner trim,12963,rear panel,rear panel,0,0,0,1,1,1
7186267,12606779,firewall rear trim,225.00,,I,rear panel inner trim,rear panel inner trim,12963,rear panel,rear panel,0,0,0,1,1,1


In [148]:
partlistLabelDf = partlistLabelDf[~partlistLabelDf["Description"].str.contains("|".join(blacklistComponent))]
partlistLabelDf

,CaseID,Description,fVAL,DamageCond,CoType,FuzzyMatchDesc,lvl_1_desc,count,lvl_2_desc,lvl_3_desc,front_view,front_view_left,front_view_right,rear_view,rear_view_left,rear_view_right
0,12657878,bonnet,550.00,bent/dented,I,bonnet,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
1,12657888,bonnet,510.44,bent,I,bonnet,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
2,12657890,bonnet,504.94,dented,I,bonnet,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
3,12657900,bonnet,450.00,crumpled,I,bonnet,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
4,12657903,bonnet,280.00,bent,I,bonnet,bonnet,233463,bonnet,bonnet,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7186264,12606104,rear panel inner trim,235.00,broken,I,rear panel inner trim,rear panel inner trim,12963,rear panel,rear panel,0,0,0,1,1,1
7186265,12606233,rear panel inner trim,350.00,dented,I,rear panel inner trim,rear panel inner trim,12963,rear panel,rear panel,0,0,0,1,1,1
7186266,12606480,rear panel inner trim,250.00,broken,I,rear panel inner trim,rear panel inner trim,12963,rear panel,rear panel,0,0,0,1,1,1
7186267,12606779,firewall rear trim,225.00,,I,rear panel inner trim,rear panel inner trim,12963,rear panel,rear panel,0,0,0,1,1,1


In [151]:

cli = boto3.client("s3", **{
    "endpoint_url": "http://192.168.1.4:8333",
    "aws_access_key_id": "",
    "aws_secret_access_key": "",
    # "Username": "aaa",
})
outputBucketName = "partlist_label"
# cli.create_bucket(Bucket=outputBucketName)
wr.s3.to_parquet(
                    df=partlistLabelDf,
                    path=f"s3://{outputBucketName}/",
                    dataset=True,
                    mode="overwrite",
                )


{'paths': ['s3://partlist_label/7c85dcd0762b4ce4aea80676eb674c57.snappy.parquet'],
 'partitions_values': {}}

In [150]:

# wr.s3.read_parquet(
#     path=f"s3://{outputBucketName}/"
# )